#Custom Neural Network

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# 필요한 라이브러리 불러오기

import os
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
# feature의 코사인 유사도 matrix 계산 함수

def cos_sim(data):

    data_similarity = np.array([])
    fst_axis = len(data.keys())
    scd_axis = len(data)

    for i in range(0, len(data.keys())-1):
        for j in range(i+1, len(data.keys())):
            info = data.iloc[:, [i, j]]
            data_sim = cosine_similarity(info)
            data_sim = (data_sim - data_sim.min())/(data_sim.max() - data_sim.min())
            data_sim = pd.DataFrame(data = data_sim,index = data.index,columns = data.index)
            data_similarity = np.append(data_similarity, np.array(data_sim)) 
    data_similarity = data_similarity.reshape(math.comb(fst_axis, 2), scd_axis, -1)
    return data_similarity

In [9]:
# 유사도 값과 실제 실험 점수 값 가중치 평균 함수
def weight_avg(sim_matx, test_score_matx):

    input_data = []

    for k in range(sim_matx.shape[0]): # 1
        line = []
        for i in range(sim_matx.shape[1]): # 37   
            sum = 0
            mean = 0
            for j in range(sim_matx.shape[2]): # 37
                if i==j:
                    continue
                else:
                    sum += sim_matx[k][i][j] * test_score_matx[j]
            line.append(sum/sim_matx.shape[1])
            #print(len(line))
        input_data.append(line)

    return input_data


In [10]:
# SNSB 테스트 점수 별 feature 정보 전처리
def data_preprocessing(data_path, y_path, test_num):

    data = pd.read_csv(data_path)
    test_score = pd.read_csv(y_path)

    if test_num == 1:
        data = data[["panel6","password_error","panel7_head_distance","panel7_hand_distance"]]
        test_score = test_score.loc[:36,"DST_F+B"]
    elif test_num == 2:
        data = data[["total_time", "password_error"]]
        test_score = test_score.loc[:36,"S-K-BNT"]
    elif test_num == 3:
        data = data[["total_time", "password_error","panel1Target_var_x","panel1Target_var_y"]]
        test_score = test_score.loc[:36,"RCFT_copyscore"]
    elif test_num == 4:
        data = data[["total_time","correct","password_error","head_velocity","hand_velocity"]]
        test_score = test_score.loc[:36,"SVLT_delayedrecall"]
    elif test_num == 5:
        data = data[["panel1", "panel2", "panel3", "panel4", "panel5", "panel6", "total_time"]]
        test_score = test_score.loc[:36,"K-TMT-E_B"]

    return data, test_score

In [11]:
# 데이터 로드
data_path = "/content/drive/MyDrive/딥러닝/딥러닝 팀 프로젝트 배포 VEEM 데이터/final_result.csv"
y_path = "/content/drive/MyDrive/딥러닝/딥러닝 팀 프로젝트 배포 VEEM 데이터/VEEM SNSB 데이터.csv"


# 데이터 전처리
# test_num 바꿔가면서 설정해주세요!
# 1 : DST_F+B
# 2 : S-K-BNT
# 3 : RCFT_copyscore
# 4 : SVLT_delatedrecall
# 5 : K-TMT-E_B
test_num=4
data, test_score = data_preprocessing(data_path,y_path,test_num)

x_train, x_test, y_train, y_test = train_test_split(data, test_score, test_size=0.1, random_state=777)
y_train = np.array(y_train)
y_test = np.array(y_test)


# train data 전처리
data_similarity = cos_sim(x_train)
print(data_similarity.shape)
x_train = np.array(weight_avg(data_similarity, y_train))
print(x_train.shape)


# test data 전처리
data_similarity = cos_sim(x_test)
print(data_similarity.shape)
x_test = np.array(weight_avg(data_similarity, y_test))
print(x_test.shape)

(10, 33, 33)
(10, 33)
(10, 4, 4)
(10, 4)


In [12]:
print(x_train.shape)
print(y_train.shape)
print(x_train.T)
print(y_train)

(10, 33)
(33,)
[[3.80804556 3.94319383 3.83075123 3.75712925 3.81045666 3.32695555
  3.78186231 2.81226363 2.98593233 3.63129949]
 [2.38501062 3.58162228 3.802637   3.17229121 3.62863848 3.09662776
  3.76411415 2.62800437 2.8925773  3.0794413 ]
 [3.31435858 4.02387912 3.83581042 3.48103076 2.99452143 3.93579107
  3.98666697 2.91549872 2.89513138 3.56568846]
 [3.34436223 3.38190853 3.95052279 3.50626147 2.06933946 3.55605803
  4.0080444  2.91214392 2.8742423  2.58713925]
 [3.25175761 3.75455357 2.14078703 3.14040097 3.58666812 3.81453278
  3.9938998  1.42727518 1.74868612 3.94983142]
 [3.01373966 3.70744165 3.69474331 3.23157669 3.64639347 2.49307315
  3.77045683 2.66322677 2.70726764 2.12931748]
 [3.5933197  3.44624594 3.89714208 3.59068375 3.29191557 3.83117516
  3.87688517 2.79525795 2.79257229 3.82628701]
 [3.62084848 3.99837365 3.98429085 3.53853711 3.722843   3.96166822
  3.76654828 2.92314124 2.90552531 2.74628042]
 [3.71762995 1.534088   3.82652533 3.17619316 3.78262669 3.926699

In [13]:
# 모델 layer 

model = Sequential()
model.add(Dense(64, input_shape=(x_test.shape[0],), activation='elu'))
model.add(Dense(32, activation='elu'))
model.add(Dense(16,  activation='elu'))
model.add(Dense(8,  activation='elu'))
model.add(Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                704       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 8)                 136       
                                                                 
 dense_4 (Dense)             (None, 1)                 9         
                                                                 
Total params: 3,457
Trainable params: 3,457
Non-trainable params: 0
_________________________________________________________________


In [14]:
# 모델 컴파일

model.compile(optimizer='adam', loss='mse', metrics='mse')

In [15]:
# 모델 학습

model.fit(np.array(x_train.T), np.array(y_train), epochs=400)

Epoch 1/400
2/2 [==============================] - 1s 16ms/step - loss: 29.1864 - mse: 29.1864
Epoch 2/400
2/2 [==============================] - 0s 6ms/step - loss: 22.7605 - mse: 22.7605
Epoch 3/400
2/2 [==============================] - 0s 6ms/step - loss: 18.3070 - mse: 18.3070
Epoch 4/400
2/2 [==============================] - 0s 9ms/step - loss: 15.6052 - mse: 15.6052
Epoch 5/400
2/2 [==============================] - 0s 7ms/step - loss: 14.1075 - mse: 14.1075
Epoch 6/400
2/2 [==============================] - 0s 10ms/step - loss: 13.0486 - mse: 13.0486
Epoch 7/400
2/2 [==============================] - 0s 7ms/step - loss: 12.3008 - mse: 12.3008
Epoch 8/400
2/2 [==============================] - 0s 4ms/step - loss: 11.7965 - mse: 11.7965
Epoch 9/400
2/2 [==============================] - 0s 4ms/step - loss: 11.5249 - mse: 11.5249
Epoch 10/400
2/2 [==============================] - 0s 8ms/step - loss: 11.4163 - mse: 11.4163
Epoch 11/400
2/2 [==============================] - 0s 5m

In [16]:
# 모델 저장
model_save_path = "/content/drive/MyDrive/딥러닝/딥러닝 팀 프로젝트 배포 VEEM 데이터/regression_" + str(test_num) + ".h5"
model.save(model_save_path)
new_model = tf.keras.models.load_model(model_save_path)


In [17]:
# model evaluation
loss, mse = new_model.evaluate(x_test.T, y_test)
print('acc : ', mse)

1/1 [==============================] - 0s 139ms/step - loss: 7.3926 - mse: 7.3926
acc :  7.392573833465576


In [18]:
y_predict = model.predict(x_test.T)
print(y_predict)
print(y_test)

1/1 [==============================] - 0s 98ms/step
[[5.795087 ]
 [5.8561563]
 [5.337141 ]
 [4.024638 ]]
[4. 1. 7. 4.]
